Carregando o dataset Brain

https://www.med.upenn.edu/sbia/brats2019/data.html

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

import os
os.chdir("/content/gdrive/My Drive/databrain")


Mounted at /content/gdrive


In [2]:
# Listar arquivos na pasta imagesTr
import os
os.chdir("/content/gdrive/My Drive/databrain/imagesTr")
files_imagesTr = os.listdir()
print("Arquivos na pasta imagesTr:")
print(files_imagesTr)

# Listar arquivos na pasta imagesTs
os.chdir("/content/gdrive/My Drive/databrain/imagesTs")
files_imagesTs = os.listdir()
print("\nArquivos na pasta imagesTs:")
print(files_imagesTs)

# Listar arquivos na pasta labelsTr
os.chdir("/content/gdrive/My Drive/databrain/labelsTr")
files_labelsTr = os.listdir()
print("\nArquivos na pasta labelsTr:")
print(files_labelsTr)


Arquivos na pasta imagesTr:
['BRATS_001.nii.gz', 'BRATS_002.nii.gz', 'BRATS_003.nii.gz', 'BRATS_004.nii.gz', 'BRATS_005.nii.gz', 'BRATS_006.nii.gz', 'BRATS_007.nii.gz', 'BRATS_008.nii.gz', 'BRATS_009.nii.gz', 'BRATS_010.nii.gz', 'BRATS_011.nii.gz', 'BRATS_012.nii.gz', 'BRATS_013.nii.gz', 'BRATS_014.nii.gz', 'BRATS_015.nii.gz', 'BRATS_016.nii.gz', 'BRATS_017.nii.gz', 'BRATS_018.nii.gz', 'BRATS_019.nii.gz', 'BRATS_020.nii.gz', 'BRATS_021.nii.gz', 'BRATS_022.nii.gz', 'BRATS_023.nii.gz', 'BRATS_024.nii.gz', 'BRATS_025.nii.gz', 'BRATS_026.nii.gz', 'BRATS_027.nii.gz', 'BRATS_028.nii.gz', 'BRATS_029.nii.gz', 'BRATS_030.nii.gz', 'BRATS_031.nii.gz', 'BRATS_032.nii.gz', 'BRATS_033.nii.gz', 'BRATS_034.nii.gz', 'BRATS_035.nii.gz', 'BRATS_036.nii.gz', 'BRATS_037.nii.gz', 'BRATS_038.nii.gz', 'BRATS_039.nii.gz', 'BRATS_040.nii.gz', 'BRATS_041.nii.gz', 'BRATS_042.nii.gz', 'BRATS_043.nii.gz', 'BRATS_044.nii.gz', 'BRATS_045.nii.gz', 'BRATS_046.nii.gz', 'BRATS_047.nii.gz', 'BRATS_048.nii.gz', 'BRATS_049.


Normaliza os valores dos pixels para que os dados fiquem na mesma escala
---Função para exibir as imagens



In [ ]:
import nibabel as nib
import numpy as np

# Defina os caminhos para as pastas com imagens e rótulos
images_dir = "/content/gdrive/My Drive/databrain/imagesTr"
labels_dir = "/content/gdrive/My Drive/databrain/labelsTr"

# Loop através dos arquivos BRATS_001.nii.gz até BRATS_484.nii.gz
for i in range(1, 485):
    # Gere o nome do arquivo a partir do índice, com preenchimento de zero (por exemplo, BRATS_001.nii.gz)
    file_index = str(i).zfill(3)
    image_filename = f"{images_dir}/BRATS_{file_index}.nii.gz"
    label_filename = f"{labels_dir}/BRATS_{file_index}.nii.gz"

    # Carregue a imagem e o rótulo
    image = nib.load(image_filename).get_fdata()
    label = nib.load(label_filename).get_fdata()

    # Pré-processamento (normalização, redimensionamento, segmentação, etc.) aqui
    # Substitua esta parte pelo seu código de pré-processamento específico.

    # Exemplo: Normalização min-max
    min_value = np.min(image)
    max_value = np.max(image)
    normalized_image = (image - min_value) / (max_value - min_value)

    # Salve a imagem pré-processada se desejar
    preprocessed_image = nib.Nifti1Image(normalized_image, nib.load(image_filename).affine)
    output_filename = f"/content/gdrive/My Drive/databrain/preprocessed_images/BRATS_{file_index}_preprocessed.nii.gz"
    nib.save(preprocessed_image, output_filename)

    # Continue com o processamento ou análise dos dados conforme necessário


In [ ]:
!pip install tensorflow
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow.keras as keras


# Defina a arquitetura do modelo UNet
def unet_model(input_shape):
    inputs = keras.Input(shape=input_shape)

    # Codificador
    conv1 = layers.Conv2D(64, 3, activation='relu', padding='same')(inputs)
    conv1 = layers.Conv2D(64, 3, activation='relu', padding='same')(conv1)
    pool1 = layers.MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = layers.Conv2D(128, 3, activation='relu', padding='same')(pool1)
    conv2 = layers.Conv2D(128, 3, activation='relu', padding='same')(conv2)
    pool2 = layers.MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = layers.Conv2D(256, 3, activation='relu', padding='same')(pool2)
    conv3 = layers.Conv2D(256, 3, activation='relu', padding='same')(conv3)
    pool3 = layers.MaxPooling2D(pool_size=(2, 2))(conv3)

    # Decodificador
    up4 = layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(conv3)
    up4 = layers.concatenate([up4, conv2], axis=-1)
    conv4 = layers.Conv2D(128, 3, activation='relu', padding='same')(up4)
    conv4 = layers.Conv2D(128, 3, activation='relu', padding='same')(conv4)

    up5 = layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(conv4)
    up5 = layers.concatenate([up5, conv1], axis=-1)
    conv5 = layers.Conv2D(64, 3, activation='relu', padding='same')(up5)
    conv5 = layers.Conv2D(64, 3, activation='relu', padding='same')(conv5)

    outputs = layers.Conv2D(1, 1, activation='sigmoid')(conv5)

    model = keras.Model(inputs, outputs)
    return model

# Crie uma instância do modelo
input_shape = (128, 128, 1)  # Ajuste o tamanho da imagem conforme necessário
model = unet_model(input_shape)

# Compile o modelo
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Exiba o resumo do modelo
model.summary()


In [ ]:
import tensorflow as tf
from tensorflow import keras

# Defina o modelo
model = keras.Sequential([
    keras.layers.Dense(64, activation='relu', input_shape=(100,)),
    keras.layers.Dense(10, activation='softmax')
])

# Compile o modelo
model.compile(optimizer='adam',  # Otimizador
              loss='categorical_crossentropy',  # Função de perda
              metrics=['accuracy'])  # Métricas para avaliação

# Para um modelo de classificação binária, você pode usar:
# loss='binary_crossentropy' e métricas=['accuracy']

# Para modelos de regressão, a função de perda e as métricas variam de acordo com o problema.

# Resumo do modelo
model.summary()


In [ ]:
import matplotlib.pyplot as plt
import nibabel as nib
import numpy as np

# Defina a pasta com as imagens pré-processadas
preprocessed_dir = "/content/gdrive/My Drive/databrain/preprocessed_images"

# Lista de índices das imagens que você deseja exibir
indices_to_display = [1, 2, 3]  # Substitua pelos índices desejados

# Função para exibir imagens
def display_images(image_paths):
    for image_path in image_paths:
        image = nib.load(image_path).get_fdata()
        plt.figure(figsize=(6, 6))
        plt.imshow(image[:, :, 0], cmap='gray')  # Exibindo a primeira fatia da imagem 3D
        plt.title(f"Imagem: {image_path.split('/')[-1]}")
        plt.axis('off')
        plt.show()

# Crie a lista de caminhos para as imagens pré-processadas
image_paths = [f"{preprocessed_dir}/BRATS_{str(i).zfill(3)}_preprocessed.nii.gz" for i in indices_to_display]

# Exiba as imagens
display_images(image_paths)


In [ ]:
# Listar arquivos na pasta labelsTr
os.chdir("/content/gdrive/My Drive/databrain/labelsTr")
files_labelsTr = os.listdir()
print("\nArquivos na pasta labelsTr:")
print(files_labelsTr)


In [ ]:
import os
import nibabel as nib
import numpy as np

# Defina os caminhos para as pastas com imagens e rótulos
images_dir = "/content/gdrive/My Drive/databrain/imagesTr"
labels_dir = "/content/gdrive/My Drive/databrain/labelsTr"

# Lista para armazenar as imagens e rótulos
images = []
labels = []

# Loop através dos arquivos BRATS_001.nii.gz até BRATS_484.nii.gz
for i in range(1, 485):
    # Gere o nome do arquivo a partir do índice, com preenchimento de zero (por exemplo, BRATS_001.nii.gz)
    file_index = str(i).zfill(3)
    image_filename = f"{images_dir}/BRATS_{file_index}.nii.gz"
    label_filename = f"{labels_dir}/BRATS_{file_index}.nii.gz"

    # Carregue a imagem e o rótulo
    image = nib.load(image_filename).get_fdata()
    label = nib.load(label_filename).get_fdata()

    # Adicione a imagem e o rótulo à lista
    images.append(image)
    labels.append(label)

# Converta as listas em arrays numpy
images = np.array(images)
labels = np.array(labels)

# Agora você pode usar as variáveis 'images' e 'labels' para treinar e validar seu modelo.


In [ ]:
historico = model.fit(preprocessed_image, labelsTr, epochs=10, validation_data=(imageTs, None))